<h1>Exploratory Data Analysis (EDA)</h1>

<p>This is the next component of the NLP pipeline (follows data cleaning). EDA is carried out to verify that the data we have makes sense. If it doesn't, then we'll have to go back to our data cleaning notebook and implement a few more techniques to transform the data into a more usable format.</p>

In [1]:
# Reading in clean data
import pandas as pd

headlines = pd.read_pickle("./pickles/headline_dtm.pkl")
sentences = pd.read_pickle("./pickles/sentence_dtm.pkl")

# Transposing the datasets for easier comprehension
headlines = headlines.transpose()
sentences = sentences.transpose()

<p>EDA is done by visualizing aspects of the cleaned dataset.</p>

<p><i>(For each visualization that's presented, there will be two variants - headlines and first sentences.)</i></p>